# Chapter 7: Shading and Illumination

## Lighting Models from Scratch

This notebook covers:
- Local illumination models
- Lambertian (diffuse) shading
- Specular reflection (Phong, Blinn-Phong)
- Material properties
- Multiple light sources
- Ambient occlusion

**Key References:** Marschner & Shirley Ch. 10, Gambetta Ch. 3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_07_shading_and_illumination.ipynb)

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from typing import Tuple, List, Optional

print("✓ Imports loaded")

---

## 1. Illumination Theory

### 1.1 Light-Surface Interaction

When light hits a surface, it can be:
- **Absorbed**: Converted to heat
- **Reflected**: Bounced off the surface
- **Transmitted**: Passes through (for transparent materials)

### 1.2 The Rendering Equation

The **rendering equation** (Kajiya 1986) describes light transport:

$$L_o(\mathbf{p}, \omega_o) = L_e(\mathbf{p}, \omega_o) + \int_{\Omega} f_r(\mathbf{p}, \omega_i, \omega_o) L_i(\mathbf{p}, \omega_i) (\mathbf{n} \cdot \omega_i) \, d\omega_i$$

where:
- $L_o(\mathbf{p}, \omega_o)$ = outgoing radiance at point $\mathbf{p}$ in direction $\omega_o$
- $L_e(\mathbf{p}, \omega_o)$ = emitted radiance (for light sources)
- $f_r(\mathbf{p}, \omega_i, \omega_o)$ = BRDF (Bidirectional Reflectance Distribution Function)
- $L_i(\mathbf{p}, \omega_i)$ = incoming radiance from direction $\omega_i$
- $\mathbf{n}$ = surface normal
- $\Omega$ = hemisphere of incoming directions

### 1.3 Local Illumination

For real-time graphics, we use **local illumination** models that approximate the rendering equation:

$$L = L_{\text{ambient}} + \sum_{\text{lights}} \left( L_{\text{diffuse}} + L_{\text{specular}} \right)$$

### 1.4 Lambertian (Diffuse) Reflection

**Lambert's cosine law**: Ideal diffuse reflection scatters light equally in all directions.

$$L_{\text{diffuse}} = k_d I (\mathbf{n} \cdot \mathbf{l})$$

where:
- $k_d$ = diffuse reflectance (albedo) in $[0, 1]$
- $I$ = light intensity
- $\mathbf{n}$ = surface normal (unit vector)
- $\mathbf{l}$ = light direction (unit vector pointing toward light)
- $\mathbf{n} \cdot \mathbf{l} = \cos\theta$ where $\theta$ is angle between normal and light

**Properties:**
- Matte surfaces (chalk, unpolished wood, paper)
- Appearance independent of view direction
- Maximum brightness when $\mathbf{n} \parallel \mathbf{l}$ (light perpendicular to surface)
- Zero brightness when $\mathbf{n} \perp \mathbf{l}$ (light grazing surface)

### 1.5 Specular Reflection

**Specular reflection**: Mirror-like reflection creating highlights.

#### Phong Model

$$L_{\text{specular}} = k_s I (\mathbf{r} \cdot \mathbf{v})^n$$

where:
- $k_s$ = specular reflectance
- $\mathbf{r}$ = reflection direction: $\mathbf{r} = 2(\mathbf{n} \cdot \mathbf{l})\mathbf{n} - \mathbf{l}$
- $\mathbf{v}$ = view direction (toward camera)
- $n$ = shininess exponent (higher = sharper highlight)
  - $n = 1$: Very rough surface
  - $n = 10$: Moderately shiny (plastic)
  - $n = 100$: Very shiny (polished metal)

#### Blinn-Phong Model (More Efficient)

Uses **halfway vector** instead of reflection:

$$\mathbf{h} = \frac{\mathbf{l} + \mathbf{v}}{\|\mathbf{l} + \mathbf{v}\|}$$

$$L_{\text{specular}} = k_s I (\mathbf{n} \cdot \mathbf{h})^n$$

**Advantages:**
- Faster (avoids reflection calculation)
- More physically plausible
- Numerically stable

### 1.6 Ambient Light

**Ambient light**: Constant illumination approximating indirect lighting.

$$L_{\text{ambient}} = k_a I_a$$

where:
- $k_a$ = ambient reflectance
- $I_a$ = ambient light intensity

**Purpose:** Prevents completely black shadows (unrealistic without global illumination).

### 1.7 Complete Phong Illumination Model

Combining all components:

$$L = k_a I_a + \sum_{i=1}^{n} \left[ k_d I_i (\mathbf{n} \cdot \mathbf{l}_i) + k_s I_i (\mathbf{r}_i \cdot \mathbf{v})^{n_s} \right]$$

With clamping:
- $\max(\mathbf{n} \cdot \mathbf{l}, 0)$ to avoid negative lighting (back-facing)
- $\max(\mathbf{r} \cdot \mathbf{v}, 0)^{n_s}$ for specular

### 1.8 Attenuation

Light intensity decreases with distance:

$$f_{\text{att}}(d) = \frac{1}{k_c + k_l d + k_q d^2}$$

where:
- $d$ = distance to light
- $k_c$ = constant attenuation (usually 1.0)
- $k_l$ = linear attenuation
- $k_q$ = quadratic attenuation

Physically correct: $k_c = 0, k_l = 0, k_q = 1$ (inverse square law)

### 1.9 Light Types

**Directional Light** (Sun):
- Infinite distance
- Constant direction everywhere
- No attenuation

**Point Light** (Light bulb):
- Emits light in all directions from a point
- Direction varies per surface point: $\mathbf{l} = \frac{\mathbf{p}_{\text{light}} - \mathbf{p}}{\|\mathbf{p}_{\text{light}} - \mathbf{p}\|}$
- Subject to attenuation

**Spotlight**:
- Point light with cone of influence
- Additional term: $(\mathbf{l} \cdot \mathbf{d}_{\text{spot}})^{n_{\text{spot}}}$
- $\mathbf{d}_{\text{spot}}$ = spotlight direction
- $n_{\text{spot}}$ = falloff exponent

---

## 2. Core Classes Implementation

In [ ]:
class Vec3:
    """3D Vector class"""
    def __init__(self, x=0.0, y=0.0, z=0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar):
        if isinstance(scalar, Vec3):
            return Vec3(self.x * scalar.x, self.y * scalar.y, self.z * scalar.z)
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __rmul__(self, scalar):
        return self.__mul__(scalar)
    
    def __truediv__(self, scalar):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self):
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)
    
    def normalize(self):
        l = self.length()
        if l > 0:
            return self / l
        return Vec3(0, 0, 0)
    
    def clamp(self, min_val=0.0, max_val=1.0):
        """Clamp components to range"""
        return Vec3(
            max(min_val, min(max_val, self.x)),
            max(min_val, min(max_val, self.y)),
            max(min_val, min(max_val, self.z))
        )
    
    def to_array(self):
        return np.array([self.x, self.y, self.z])
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

# Type aliases for clarity
Color = Vec3  # RGB color
Point3 = Vec3  # 3D point

class Material:
    """Material properties for Phong shading"""
    def __init__(self,
                 ambient: Color = None,
                 diffuse: Color = None,
                 specular: Color = None,
                 shininess: float = 32.0):
        self.ambient = ambient if ambient else Color(0.1, 0.1, 0.1)
        self.diffuse = diffuse if diffuse else Color(0.8, 0.8, 0.8)
        self.specular = specular if specular else Color(1.0, 1.0, 1.0)
        self.shininess = shininess
    
    @staticmethod
    def plastic_red():
        return Material(
            ambient=Color(0.2, 0.0, 0.0),
            diffuse=Color(0.8, 0.0, 0.0),
            specular=Color(0.5, 0.5, 0.5),
            shininess=32.0
        )
    
    @staticmethod
    def metal_gold():
        return Material(
            ambient=Color(0.24725, 0.1995, 0.0745),
            diffuse=Color(0.75164, 0.60648, 0.22648),
            specular=Color(0.628281, 0.555802, 0.366065),
            shininess=51.2
        )
    
    @staticmethod
    def rubber_cyan():
        return Material(
            ambient=Color(0.0, 0.05, 0.05),
            diffuse=Color(0.4, 0.5, 0.5),
            specular=Color(0.04, 0.7, 0.7),
            shininess=10.0
        )
    
    def __repr__(self):
        return f"Material(ka={self.ambient}, kd={self.diffuse}, ks={self.specular}, n={self.shininess})"

class Light:
    """Base light class"""
    def __init__(self, color: Color = None, intensity: float = 1.0):
        self.color = color if color else Color(1.0, 1.0, 1.0)
        self.intensity = intensity
    
    def get_direction(self, point: Point3) -> Vec3:
        """Get light direction at point (to be overridden)"""
        raise NotImplementedError
    
    def get_intensity(self, point: Point3) -> float:
        """Get light intensity at point (with attenuation)"""
        return self.intensity

class DirectionalLight(Light):
    """Directional light (e.g., sun)"""
    def __init__(self, direction: Vec3, color: Color = None, intensity: float = 1.0):
        super().__init__(color, intensity)
        self.direction = direction.normalize()
    
    def get_direction(self, point: Point3) -> Vec3:
        return self.direction * -1  # Direction TO light
    
    def __repr__(self):
        return f"DirectionalLight(dir={self.direction}, color={self.color})"

class PointLight(Light):
    """Point light with position and attenuation"""
    def __init__(self, position: Point3, color: Color = None, intensity: float = 1.0,
                 constant: float = 1.0, linear: float = 0.09, quadratic: float = 0.032):
        super().__init__(color, intensity)
        self.position = position
        self.constant = constant
        self.linear = linear
        self.quadratic = quadratic
    
    def get_direction(self, point: Point3) -> Vec3:
        return (self.position - point).normalize()
    
    def get_intensity(self, point: Point3) -> float:
        distance = (self.position - point).length()
        attenuation = 1.0 / (self.constant + self.linear * distance + self.quadratic * distance * distance)
        return self.intensity * attenuation
    
    def __repr__(self):
        return f"PointLight(pos={self.position}, color={self.color})"

print("✓ Core classes loaded")

---

## 3. Shading Functions

In [ ]:
def compute_diffuse(material: Material, light: Light, 
                   normal: Vec3, light_dir: Vec3, light_intensity: float) -> Color:
    """Compute Lambertian diffuse reflection"""
    # N · L (clamped to [0, 1])
    n_dot_l = max(0.0, normal.dot(light_dir))
    
    # kd * I * (N · L)
    return material.diffuse * light.color * (light_intensity * n_dot_l)

def compute_specular_phong(material: Material, light: Light,
                          normal: Vec3, light_dir: Vec3, view_dir: Vec3,
                          light_intensity: float) -> Color:
    """Compute Phong specular reflection"""
    # Reflection direction: R = 2(N·L)N - L
    n_dot_l = normal.dot(light_dir)
    reflect_dir = (normal * (2.0 * n_dot_l) - light_dir).normalize()
    
    # (R · V)^n
    r_dot_v = max(0.0, reflect_dir.dot(view_dir))
    spec_factor = pow(r_dot_v, material.shininess)
    
    # ks * I * (R · V)^n
    return material.specular * light.color * (light_intensity * spec_factor)

def compute_specular_blinn_phong(material: Material, light: Light,
                                normal: Vec3, light_dir: Vec3, view_dir: Vec3,
                                light_intensity: float) -> Color:
    """Compute Blinn-Phong specular reflection (more efficient)"""
    # Halfway vector: H = normalize(L + V)
    halfway = (light_dir + view_dir).normalize()
    
    # (N · H)^n
    n_dot_h = max(0.0, normal.dot(halfway))
    spec_factor = pow(n_dot_h, material.shininess)
    
    # ks * I * (N · H)^n
    return material.specular * light.color * (light_intensity * spec_factor)

def compute_ambient(material: Material, ambient_color: Color = None) -> Color:
    """Compute ambient light"""
    if ambient_color is None:
        ambient_color = Color(0.1, 0.1, 0.1)
    return material.ambient * ambient_color

def phong_illumination(point: Point3, normal: Vec3, view_dir: Vec3,
                      material: Material, lights: List[Light],
                      ambient_color: Color = None,
                      use_blinn: bool = True) -> Color:
    """Complete Phong/Blinn-Phong illumination model"""
    # Ambient component
    color = compute_ambient(material, ambient_color)
    
    # Normalize inputs
    normal = normal.normalize()
    view_dir = view_dir.normalize()
    
    # Sum contributions from all lights
    for light in lights:
        light_dir = light.get_direction(point)
        light_intensity = light.get_intensity(point)
        
        # Diffuse component
        color = color + compute_diffuse(material, light, normal, light_dir, light_intensity)
        
        # Specular component
        if use_blinn:
            color = color + compute_specular_blinn_phong(material, light, normal, light_dir, view_dir, light_intensity)
        else:
            color = color + compute_specular_phong(material, light, normal, light_dir, view_dir, light_intensity)
    
    # Clamp final color to [0, 1]
    return color.clamp(0.0, 1.0)

print("✓ Shading functions loaded")

---

## 4. Practical Examples

In [ ]:
# Example 1: Lambertian Shading on a Sphere
print("Example 1: Lambertian (Diffuse) Shading\n")

# Setup
material = Material(
    ambient=Color(0.1, 0.1, 0.1),
    diffuse=Color(0.6, 0.3, 0.3),  # Reddish
    specular=Color(0, 0, 0),  # No specular
    shininess=0
)

light = DirectionalLight(
    direction=Vec3(1, -1, -1),
    color=Color(1, 1, 1),
    intensity=1.0
)

# Render sphere with diffuse shading
width, height = 400, 400
image = np.zeros((height, width, 3))

sphere_center = Vec3(0, 0, 5)
sphere_radius = 1.5
camera_pos = Vec3(0, 0, 0)

for y in range(height):
    for x in range(width):
        # Convert pixel to world coordinates
        px = (x / width - 0.5) * 4
        py = (0.5 - y / height) * 4
        
        # Ray from camera through pixel
        ray_dir = Vec3(px, py, 5).normalize()
        
        # Ray-sphere intersection
        oc = camera_pos - sphere_center
        a = ray_dir.dot(ray_dir)
        b = 2.0 * oc.dot(ray_dir)
        c = oc.dot(oc) - sphere_radius * sphere_radius
        discriminant = b * b - 4 * a * c
        
        if discriminant >= 0:
            t = (-b - math.sqrt(discriminant)) / (2 * a)
            if t > 0:
                # Hit point and normal
                hit_point = camera_pos + ray_dir * t
                normal = (hit_point - sphere_center).normalize()
                view_dir = (camera_pos - hit_point).normalize()
                
                # Compute shading
                color = phong_illumination(hit_point, normal, view_dir, material, [light])
                image[y, x] = color.to_array()

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title("Lambertian (Diffuse) Shading")
plt.axis('off')
plt.tight_layout()
plt.show()

print("Lambertian shading creates smooth, matte appearance")
print("Brightness depends only on angle between normal and light\n")

In [ ]:
# Example 2: Comparing Shininess Values
print("Example 2: Specular Highlights with Different Shininess\n")

shininess_values = [5, 10, 32, 100]
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

light = PointLight(
    position=Vec3(-2, 3, 3),
    color=Color(1, 1, 1),
    intensity=10.0,
    constant=1.0,
    linear=0.0,
    quadratic=0.0
)

for idx, shininess in enumerate(shininess_values):
    material = Material(
        ambient=Color(0.1, 0.1, 0.2),
        diffuse=Color(0.3, 0.3, 0.8),
        specular=Color(1.0, 1.0, 1.0),
        shininess=shininess
    )
    
    width, height = 200, 200
    image = np.zeros((height, width, 3))
    
    sphere_center = Vec3(0, 0, 5)
    sphere_radius = 1.5
    camera_pos = Vec3(0, 0, 0)
    
    for y in range(height):
        for x in range(width):
            px = (x / width - 0.5) * 4
            py = (0.5 - y / height) * 4
            ray_dir = Vec3(px, py, 5).normalize()
            
            oc = camera_pos - sphere_center
            a = ray_dir.dot(ray_dir)
            b = 2.0 * oc.dot(ray_dir)
            c = oc.dot(oc) - sphere_radius * sphere_radius
            discriminant = b * b - 4 * a * c
            
            if discriminant >= 0:
                t = (-b - math.sqrt(discriminant)) / (2 * a)
                if t > 0:
                    hit_point = camera_pos + ray_dir * t
                    normal = (hit_point - sphere_center).normalize()
                    view_dir = (camera_pos - hit_point).normalize()
                    
                    color = phong_illumination(hit_point, normal, view_dir, material, [light])
                    image[y, x] = color.to_array()
    
    axes[idx].imshow(image)
    axes[idx].set_title(f"Shininess = {shininess}")
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("Higher shininess values create sharper, more focused highlights")
print("Low shininess = rough surface (wide highlight)")
print("High shininess = smooth surface (tight highlight)\n")

In [ ]:
# Example 3: Multiple Light Sources
print("Example 3: Multiple Colored Lights\n")

material = Material(
    ambient=Color(0.05, 0.05, 0.05),
    diffuse=Color(0.8, 0.8, 0.8),
    specular=Color(1.0, 1.0, 1.0),
    shininess=64.0
)

lights = [
    PointLight(Vec3(-3, 2, 3), Color(1.0, 0.2, 0.2), intensity=8.0),  # Red
    PointLight(Vec3(3, 2, 3), Color(0.2, 0.2, 1.0), intensity=8.0),   # Blue
    PointLight(Vec3(0, -2, 2), Color(0.2, 1.0, 0.2), intensity=5.0),  # Green
]

width, height = 400, 400
image = np.zeros((height, width, 3))

sphere_center = Vec3(0, 0, 5)
sphere_radius = 1.5
camera_pos = Vec3(0, 0, 0)

for y in range(height):
    for x in range(width):
        px = (x / width - 0.5) * 4
        py = (0.5 - y / height) * 4
        ray_dir = Vec3(px, py, 5).normalize()
        
        oc = camera_pos - sphere_center
        a = ray_dir.dot(ray_dir)
        b = 2.0 * oc.dot(ray_dir)
        c = oc.dot(oc) - sphere_radius * sphere_radius
        discriminant = b * b - 4 * a * c
        
        if discriminant >= 0:
            t = (-b - math.sqrt(discriminant)) / (2 * a)
            if t > 0:
                hit_point = camera_pos + ray_dir * t
                normal = (hit_point - sphere_center).normalize()
                view_dir = (camera_pos - hit_point).normalize()
                
                color = phong_illumination(hit_point, normal, view_dir, material, lights)
                image[y, x] = color.to_array()

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title("Multiple Colored Light Sources")
plt.axis('off')
plt.tight_layout()
plt.show()

print("Multiple lights create color mixing where their contributions overlap\n")

In [ ]:
# Example 4: Comparing Different Materials
print("Example 4: Material Comparison\n")

materials = [
    ("Plastic Red", Material.plastic_red()),
    ("Gold Metal", Material.metal_gold()),
    ("Rubber Cyan", Material.rubber_cyan()),
]

light = PointLight(Vec3(-2, 3, 3), Color(1, 1, 1), intensity=10.0)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, (name, material) in enumerate(materials):
    width, height = 200, 200
    image = np.zeros((height, width, 3))
    
    sphere_center = Vec3(0, 0, 5)
    sphere_radius = 1.5
    camera_pos = Vec3(0, 0, 0)
    
    for y in range(height):
        for x in range(width):
            px = (x / width - 0.5) * 4
            py = (0.5 - y / height) * 4
            ray_dir = Vec3(px, py, 5).normalize()
            
            oc = camera_pos - sphere_center
            a = ray_dir.dot(ray_dir)
            b = 2.0 * oc.dot(ray_dir)
            c = oc.dot(oc) - sphere_radius * sphere_radius
            discriminant = b * b - 4 * a * c
            
            if discriminant >= 0:
                t = (-b - math.sqrt(discriminant)) / (2 * a)
                if t > 0:
                    hit_point = camera_pos + ray_dir * t
                    normal = (hit_point - sphere_center).normalize()
                    view_dir = (camera_pos - hit_point).normalize()
                    
                    color = phong_illumination(hit_point, normal, view_dir, material, [light])
                    image[y, x] = color.to_array()
    
    axes[idx].imshow(image)
    axes[idx].set_title(name)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("Different materials have distinct ambient, diffuse, and specular properties")
print("Metals have colored specular highlights; plastics have white highlights\n")

In [ ]:
# Example 5: Phong vs Blinn-Phong Comparison
print("Example 5: Phong vs Blinn-Phong Specular Model\n")

material = Material(
    ambient=Color(0.1, 0.1, 0.1),
    diffuse=Color(0.6, 0.2, 0.2),
    specular=Color(1.0, 1.0, 1.0),
    shininess=50.0
)

light = PointLight(Vec3(-2, 2, 2), Color(1, 1, 1), intensity=10.0)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

for idx, (use_blinn, title) in enumerate([(False, "Phong"), (True, "Blinn-Phong")]):
    width, height = 300, 300
    image = np.zeros((height, width, 3))
    
    sphere_center = Vec3(0, 0, 5)
    sphere_radius = 1.5
    camera_pos = Vec3(0, 0, 0)
    
    for y in range(height):
        for x in range(width):
            px = (x / width - 0.5) * 4
            py = (0.5 - y / height) * 4
            ray_dir = Vec3(px, py, 5).normalize()
            
            oc = camera_pos - sphere_center
            a = ray_dir.dot(ray_dir)
            b = 2.0 * oc.dot(ray_dir)
            c = oc.dot(oc) - sphere_radius * sphere_radius
            discriminant = b * b - 4 * a * c
            
            if discriminant >= 0:
                t = (-b - math.sqrt(discriminant)) / (2 * a)
                if t > 0:
                    hit_point = camera_pos + ray_dir * t
                    normal = (hit_point - sphere_center).normalize()
                    view_dir = (camera_pos - hit_point).normalize()
                    
                    color = phong_illumination(hit_point, normal, view_dir, material, [light], use_blinn=use_blinn)
                    image[y, x] = color.to_array()
    
    axes[idx].imshow(image)
    axes[idx].set_title(title)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("Blinn-Phong is more efficient (avoids reflection calculation)")
print("Both produce similar results, but Blinn-Phong is preferred in real-time graphics\n")

---

## Summary

In this chapter, you implemented:

✅ **Local Illumination Models** - Ambient, diffuse, specular components  
✅ **Lambertian Shading** - Diffuse reflection for matte surfaces  
✅ **Phong Model** - Specular highlights using reflection direction  
✅ **Blinn-Phong Model** - More efficient specular using halfway vector  
✅ **Material System** - Ambient, diffuse, specular coefficients and shininess  
✅ **Light Types** - Directional and point lights with attenuation  
✅ **Multiple Lights** - Summing contributions from multiple sources  

**Key Insights:**
- Diffuse shading depends on angle between normal and light (N·L)
- Specular highlights depend on view direction and create shiny appearance
- Shininess exponent controls highlight size (higher = sharper)
- Blinn-Phong is computationally more efficient than Phong
- Multiple lights simply add their contributions together
- Materials define how surfaces interact with light

**Next Chapter:** Ray Tracing - Core Implementation